# Gas turbine combustor in Cantera using constant volume reactor

Description:

What to explain in my paper:
GRI 3.0 Mechanism
PFR Reactor
Gas Turbine Combustor
How does Cantera calculate

Sources:
-Cantera documentation and code examples:
https://www.cantera.org/docs/sphinx/html/reactors.html
https://www.cantera.org/docs/sphinx/html/cython/examples/reactors_combustor.html#py-example-combustor-py
https://www.cantera.org/docs/sphinx/html/cython/examples/reactors_pfr.html#py-example-pfr-py

Importing necessery libraries

In [ ]:
import math
import csv
import cantera as ct
import numpy as np
import matplotlib.pyplot as plt

Input parameters

In [ ]:
T_0_air = 600.0  # air inlet temperature [K]
T_0_methane = 283.0 # methane inlet temparute [K]
pressure = 2500000.0  # constant pressure [Pa]
length = 5e-4  # FR length [m]
u_0 = 1.0  # inflow velocity [m/s]
area = 3.14e-4  # cross-sectional area [m**2] kolo 20 cm srednicy

# Resolution: The PFR will be simulated by 'by a chain of 'n_steps' stirred reactors.
n_steps = 10000

In [ ]:
Body

In [ ]:
# use reaction mechanism GRI-Mech 3.0

gas = ct.Solution('gri30.xml')

# create a reservoir for the fuel inlet, and set to pure methane.
gas.TPX = 300.0, ct.one_atm, 'CH4:1.0'
fuel_in = ct.Reservoir(gas)
fuel_mw = gas.mean_molecular_weight

# use predefined function Air() for the air inlet
air = ct.Solution('air.xml')
air_in = ct.Reservoir(air)
air_mw = air.mean_molecular_weight

# to ignite the fuel/air mixture, we'll introduce a pulse of radicals. The
# steady-state behavior is independent of how we do this, so we'll just use a
# stream of pure atomic hydrogen.
gas.TPX = 300.0, ct.one_atm, 'H:1.0'
igniter = ct.Reservoir(gas)

# create the combustor, and fill it in initially with N2
gas.TPX = 300.0, ct.one_atm, 'N2:1.0'
combustor = ct.IdealGasReactor(gas)
combustor.volume = 1.0

# create a reservoir for the exhaust
exhaust = ct.Reservoir(gas)

# lean combustion, phi = 0.5
equiv_ratio = 0.5

# compute fuel and air mass flow rates
factor = 0.1
air_mdot = factor * 9.52 * air_mw
fuel_mdot = factor * equiv_ratio * fuel_mw

# create and install the mass flow controllers. Controllers m1 and m2 provide
# constant mass flow rates, and m3 provides a short Gaussian pulse only to
# ignite the mixture
m1 = ct.MassFlowController(fuel_in, combustor, mdot=fuel_mdot)

# note that this connects two reactors with different reaction mechanisms and
# different numbers of species. Downstream and upstream species are matched by
# name.
m2 = ct.MassFlowController(air_in, combustor, mdot=air_mdot)

# The igniter will use a Gaussian time-dependent mass flow rate.
fwhm = 0.2
amplitude = 0.1
t0 = 1.0
igniter_mdot = lambda t: amplitude * math.exp(-(t-t0)**2 * 4 * math.log(2) / fwhm**2)
m3 = ct.MassFlowController(igniter, combustor, mdot=igniter_mdot)

# put a valve on the exhaust line to regulate the pressure
v = ct.Valve(combustor, exhaust, K=1.0)

# the simulation only contains one reactor
sim = ct.ReactorNet([combustor])


In [ ]:
Plotting